# Lab 1.3: Exploratory Data Analysis (EDA) with Pandas

## Outline

* Introduction to Pandas
* EDA using Pandas

Exploratory data analysis is a first crucial step to building predictive models from your data. EDA allows you
to confirm or invalidate some of the assumptions you are making about your data and understand relationships between your variables.

## Getting Started with [Pandas](http://pandas.pydata.org/pandas-docs/version/0.17.1/)

### What is Pandas?
A Python library providing data structures and data analysis tools.

### Huh?
Think of it like Excel for Python but better.

In [8]:
!pip install pandas
#By convention import pandas like:
import pandas as pd

You are using pip version 19.0.3, however version 19.2.3 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [9]:
#For fake data.
from numpy.random import randn
import numpy as np

## Series
Think of a Pandas Series as a _labeled_ one-dimensional vector. In fact, it need not be a numeric vector, it can contain arbitrary python objects.

In [4]:
int_series = pd.Series(range(10))
int_series.head()

0    0
1    1
2    2
3    3
4    4
dtype: int64

In [5]:
num_series = pd.Series(randn(10))
num_series.head()

0    1.351505
1    1.029232
2   -0.671339
3    1.497215
4   -0.313911
dtype: float64

In [6]:
str_series = pd.Series([x for x in 'abcde'*2])
str_series.head()

0    a
1    b
2    c
3    d
4    e
dtype: object

## Indexes
Notice how each series has an index (in this case a relatively meaningless default index).

Pandas can make great use of informative indexes. Indexes work similarly to a dictionary key, allowing fast lookups of the data associated with the index.

Indexes can also be exploited for fast group-bys, merges, time-series operations and lots more.

When you're really in the zone with pandas, you'll be thinking a lot about indexes.

In [0]:
indexed_series = pd.Series(randn(5), 
                           index = ['California', 'Alabama', 
                                    'Indiana', 'Montana', 
                                    'Kentucky'])
alt_indexed_series = pd.Series(randn(5),
                               index = ['Washington', 'Alabama', 
                                        'Montana', 'Indiana', 
                                        'New York'])
print (indexed_series)
print ('\n')
print (alt_indexed_series)

In [0]:
#Pandas uses the index by default to align series for arithmetic!
indexed_series + alt_indexed_series

In [0]:
indexed_series = pd.Series(randn(5*10), 
                           index = ['California', 'Alabama', 
                                    'Indiana', 'Montana', 
                                    'Kentucky']*10)
#If you have non-unique indexes, you can use them 
#to do groupby operations.
indexed_series.groupby(level=0).mean()

In [0]:
#Datetime index
dt_index = pd.date_range('2015-1-1', 
                        '2015-11-1', 
                        freq='m')
dt_series = pd.Series(randn(10), 
                      index = dt_index)
dt_series

In [0]:
#Datetime indexes make it easy to transform freqs etc!
dt_series.resample('q').mean()

## DataFrames
Data frames extend the concept of Series to table-like data.

In [0]:
df = pd.DataFrame(randn(10, 5), index=dt_index, columns=[x for x in 'abcde'])
df

In [0]:
#A dataframes columns are series:
col = df.a
type(col)

In [0]:
#So are the rows.
row = df.ix['2015-01-31']
type(row)

In [0]:
#The columns all have the same index:
col.index   

In [0]:
#What's the index for the rows?
row.index

## DataFrame basics

In [0]:
#New column
df['new'] = df['a'] + df['b']
df

In [0]:
#Delete a column
df.drop('new', axis=1)

## Axis?
Because pandas thinks of rows and columns as both being series, anything we can do to rows we can do to columns too. 

Axis describes which one we want to do it to. 0=rows, 1=columns.

In [0]:
lbl = pd.Timestamp('2015-04-30 00:00:00', offset='M')
df.drop(lbl, axis=0)

## Selecting subsets
There a couple of ways to select a subset of a python data frame.

In [0]:
#To subset the rows, you can use the convenient:
df[df.a > 0]

In [0]:
#Or combine multiple conditions:
df[(df.a > 0) & (df.c < 0)]

In [0]:
#Selecting a column
df.a
#Works sometimes.

In [0]:
df['a'] #works always

In [0]:
df[['a', 'b']]
#Or a subset of columns

## Advanced selection
The above methods work for simple types of selections, but what if you need something more complex?

In [0]:
df.loc['2015-05-31':'2015-08-31', 'c':'e'] #Ranges by label.

In [0]:
df.iloc[2:-3,2:5] #Ranges by number.

In [0]:
# SQL-like queries for parsimonious row selection.
# Experimental
df.query('a > 1 and c <1')

In [0]:
# Multi Index:
dt_index = pd.date_range('2015-1-1', 
                        '2017-7-1', 
                        freq='m')
df = pd.DataFrame(randn(30,5), index=dt_index)

In [0]:
df['state'] = ['Alabama', 'Alaska' , 'Arizona'] * 10
df.head()

In [0]:
df = df.reset_index()
df = df.set_index(['state', 'index'])
df.head()

In [0]:
df.loc['Alabama'].head()

In [0]:
df.loc['2015-01-31']  # Doesn't work.

In [0]:
df.loc[('Alabama', '2015-01-31')]  # Can do this.

In [0]:
# Can also have multi-index in columns.
df.reset_index().set_index(['index', 'state']).unstack().head()

In [0]:
df.head()

## split-apply-combine

In [0]:
df.groupby(level=1).mean().head()

In [0]:
df.groupby(level=0).mean().head()  # Groupby index

In [0]:
# Groupby doesn't have to be on index.
df.reset_index().groupby('state').mean().head()
# 'state' could be a list of columns to group on.

In [0]:
# Apply
g = df.groupby(level=0)

In [0]:
# If one row for each index.
g.aggregate(np.mean)

In [0]:
# If same shape as original:
g.transform(lambda x: (x - x.mean())/x.std()).head()

## Reading and writing data

In [0]:
df.to_csv('saved_data.csv', delimiter='\t')

In [0]:
read = pd.read_csv('saved_data.csv', delimiter='\t')
#TONS OF options for reading data

"""Other methods:
pd.read_excel
pd.read_sql
pd.read_stata
...
"""

## Other very useful things

join, concat and merge

In [0]:
state_df = pd.DataFrame({'governor':['Robert Bentley',
                                    'Bill Walker',
                                    'Doug Ducey',
                                    'Asa Hutchinson']}, 
                        index=['Alabama', 'Alaska', 'Arizona', 'Arkansas'])
#Note merge is most useful when you want to merge on something other than the index.
#Default is to merge on common column names.
pd.merge(df.reset_index(), state_df, 
         left_on='state', right_index=True, how='right').tail()

In [0]:
state_avg = df.groupby(level=0).mean()
state_avg.head()

In [0]:
#Concat allows joining along the axes.
pd.concat([state_avg, state_df], axis=1)

In [0]:
#Join also works. Just like merge, but 
#default is to join on indexes.
state_avg.join(state_df, how='right')

## Exploratory Data Analysis with Pandas

In [0]:
%pylab inline

In [0]:
df = pd.read_csv('https://s3-us-west-2.amazonaws.com/dsci/6002/data/playgolf.csv', delimiter='|' )
df.head()

## Describe the continuous variables
### This treats the Boolean Windy variable as a series of 0's and 1's

In [0]:
df.describe()

Can see the general pattern of Temperature and Humidity and mean of a Boolean represents the percentage

### We can make use of df.plot() to produce simple graphs that calls on the more adjustable [Matplotlib](http://matplotlib.org/api/pyplot_api.html) library 

In [0]:
df.hist(['Temperature','Humidity'],bins=5)

In [0]:
df[['Temperature','Humidity']].plot(kind='box')

### Scatterplots for examining bivariate relationships

In [0]:
df.plot('Temperature', 'Humidity', kind='scatter');

## What about the categorical variables? Frequency tables and relative frequency tables

### Simply df.value_counts() gets you the frequencies

In [0]:
df['Outlook'].value_counts()

### Using apply will get you the value counts for multiple columns at once

In [0]:
df[['Outlook','Result']].apply(lambda x: x.value_counts())

### Contingency Tables for looking at bivariate relationships between two categorical variables

In [0]:
pd.crosstab(df['Outlook'], df['Result'])

### Often we want the row percentages

In [0]:
pd.crosstab(df['Outlook'], df['Result']).apply(lambda r: r/r.sum(), axis=1)

### Or the column percentages

In [0]:
pd.crosstab(df['Outlook'], df['Result']).apply(lambda c: c/c.sum(), axis=0)

## Lab Exercises [GRADED]

In this scenario, you are a data scientist at [Bay Area Bike Share](https://www.kaggle.com/benhamner/sf-bay-area-bike-share#trip.csv). Your task
is to provide insights on bike user activity and behavior to the products team. 


1. Load the `trips` table into a dataframe. 
   
   Make 4 extra columns from the `start_date` column (We will use these in later questions):
   - `month` would contain only the month component
   - `dayofweek` would indicate what day of the week the date is
   - `date` would contain only the date component 
   - `hour` would only contain the hour component
   - [Hint to deal with datetime objects in pandas](http://stackoverflow.com/questions/25129144/pandas-return-hour-from-datetime-column-directly)

2. Group the bike rides by `month` and count the number of users per month. Plot the number of users for each month. 
   What do you observe? Provide a likely explanation to your observation. Real life data can often be messy/incomplete
   and cursory EDA is often able to reveal that.
   
3. Plot the daily user count from September to December. Mark the `mean` and `mean +/- 1.5 * Standard Deviation` as 
   horizontal lines on the plot. This would help you identify the outliers in your data. Describe your observations. 
   
   ![image](https://github.com/dannypaz/class/blob/master/dsci-6002/week1/1-3_EDA/images/timeseries.png?raw=1)

4. Plot the distribution of the daily user counts for all months as a histogram. Comment on the distribution you see and explain why the distribution might be shaped as such. 
    
   <img src="https://github.com/dannypaz/class/blob/master/dsci-6002/week1/1-3_EDA/images/hist.png?raw=1" width="500">
  
5. Now we are going to explore hourly trends of user activity. Group the bike rides by `date` and `hour` and count 
   the number of rides in the given hour on the given date. Make a 
   [boxplot](http://blog.bharatbhole.com/creating-boxplots-with-matplotlib/) of the hours in the day **(x)** against
   the number of users **(y)** in that given hour. 
   
6. Someone from the analytics team made a line plot (_right_) that he claims is showing the same information as your
   boxplot (_left_). What information can you gain from the boxplot that is missing in the line plot?
   
   ![image](https://github.com/dannypaz/class/blob/master/dsci-6002/week1/1-3_EDA/images/q1_pair.png?raw=1)

7. ** Extra Credit **: Replot the boxplot in `6.` after binning your data into weekday and weekend. Describe the differences you observe between hour user activity between weekday and weekend? 
    

# Research for tomorrow:
## What is Anscombe's Quartet?
### Be prepared to explain it tomorrow at the beginning of class.